In [1]:
import onnx
import onnxruntime as ort
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F

## 定义要处理的测试集试次ID

## 1. 读取ONNX模型

In [2]:
ID = 50
User = '10_ZZW'

In [3]:
# 加载模型
onnx_model = onnx.load('Model/1DCNN_BiLSTM.onnx')
onnx.checker.check_model(onnx_model)
# 创建 ONNX Runtime 推理会话
ort_session = ort.InferenceSession('Model/1DCNN_BiLSTM.onnx', providers=['CUDAExecutionProvider'])

## 2. 准备数据集

In [4]:
my_df = pd.read_csv('Dataset/Final_Dataset_2.csv')
my_df.head()

,User,ID,Time,Hotas_Axis_X,Hotas_Axis_Y,Hotas_Axis_Z,Hotas_Btn_Create_Fleet,Hotas_Btn_Select_UAV1,Hotas_Btn_Select_UAV2,Hotas_Btn_Select_UAV3,...,Target_Area_Pos_X,Target_Area_Pos_Y,Trapped_People_State,Trapped_People_Pos_X,Trapped_People_Pos_Y,Gaze point X,Gaze point Y,AOI,Primary Intent,Secondary Intent
0,1,1,1900-01-01 16:12:31.303,0.0,0.0,-0.307087,False,False,False,False,...,-1.0,-1.0,False,-1.0,-1.0,2067.0,1195.0,4,Intent_0,Intent_0_0
1,1,1,1900-01-01 16:12:31.335,0.0,0.0,-0.307087,False,False,False,False,...,-1.0,-1.0,False,-1.0,-1.0,2046.0,1204.0,4,Intent_0,Intent_0_0
2,1,1,1900-01-01 16:12:31.361,0.0,0.0,-0.307087,False,False,False,False,...,-1.0,-1.0,False,-1.0,-1.0,2046.0,1200.0,4,Intent_0,Intent_0_0
3,1,1,1900-01-01 16:12:31.387,0.0,0.0,-0.307087,False,False,False,False,...,-1.0,-1.0,False,-1.0,-1.0,2053.0,1221.0,4,Intent_0,Intent_0_0
4,1,1,1900-01-01 16:12:31.415,0.0,0.0,-0.307087,False,False,False,False,...,-1.0,-1.0,False,-1.0,-1.0,2068.0,1209.0,4,Intent_0,Intent_0_0


In [5]:
# 设置测试集
test_ids = [i for i in range(1, 51) if i % 5 == 0] # 测试集试次
print("test IDs:", test_ids)

df_test = my_df[my_df['ID'].isin(test_ids)]
df_test.head()

test IDs: [5, 10, 15, 20, 25, 30, 35, 40, 45, 50]


,User,ID,Time,Hotas_Axis_X,Hotas_Axis_Y,Hotas_Axis_Z,Hotas_Btn_Create_Fleet,Hotas_Btn_Select_UAV1,Hotas_Btn_Select_UAV2,Hotas_Btn_Select_UAV3,...,Target_Area_Pos_X,Target_Area_Pos_Y,Trapped_People_State,Trapped_People_Pos_X,Trapped_People_Pos_Y,Gaze point X,Gaze point Y,AOI,Primary Intent,Secondary Intent
83029,1,5,1900-01-01 14:46:17.924,0.0,0.0,-0.421107,False,False,False,False,...,-1.0,-1.0,False,-1.0,-1.0,2031.0,1247.0,0,Intent_0,Intent_0_0
83030,1,5,1900-01-01 14:46:17.949,0.0,0.0,-0.421107,False,False,False,False,...,-1.0,-1.0,False,-1.0,-1.0,2232.0,1230.0,4,Intent_0,Intent_0_0
83031,1,5,1900-01-01 14:46:17.983,0.0,0.0,-0.421107,False,False,False,False,...,-1.0,-1.0,False,-1.0,-1.0,2241.0,1210.0,4,Intent_0,Intent_0_0
83032,1,5,1900-01-01 14:46:18.011,0.0,0.0,-0.421107,False,False,False,False,...,-1.0,-1.0,False,-1.0,-1.0,2239.0,1223.0,4,Intent_1,Intent_1_1
83033,1,5,1900-01-01 14:46:18.037,0.0,0.0,-0.421107,False,False,False,False,...,-1.0,-1.0,False,-1.0,-1.0,2252.0,1209.0,4,Intent_1,Intent_1_1


### 2.1 函数：意图重编码

In [6]:
def secondary_intent_encoding(df):
    df['New Intent'] = None
    for index, row in df.iterrows():
        if row['Secondary Intent'] == 'Intent_0_0':
            df.at[index, 'New Intent'] = '00'
        if row['Secondary Intent'] == 'Intent_1_1':
            df.at[index, 'New Intent'] = '01'
        if row['Secondary Intent'] == 'Intent_1_2':
            df.at[index, 'New Intent'] = '02'
        if row['Secondary Intent'] == 'Intent_2_1' or row['Secondary Intent'] == 'Intent_3_2':
            df.at[index, 'New Intent'] = '03'
        if row['Secondary Intent'] == 'Intent_2_2':
            df.at[index, 'New Intent'] = '04'
        if row['Secondary Intent'] == 'Intent_2_3':
            df.at[index, 'New Intent'] = '05'
        if row['Secondary Intent'] == 'Intent_3_1':
            df.at[index, 'New Intent'] = '06'
        if row['Secondary Intent'] == 'Intent_3_3':
            df.at[index, 'New Intent'] = '07'
        if row['Secondary Intent'] == 'Intent_3_4':
            df.at[index, 'New Intent'] = '08'
        if row['Secondary Intent'] == 'Intent_3_5':
            df.at[index, 'New Intent'] = '09'
        if row['Secondary Intent'] == 'Intent_4_1' or row['Secondary Intent'] == 'Intent_5_1':
            df.at[index, 'New Intent'] = '10'
        if row['Secondary Intent'] == 'Intent_4_2':
            df.at[index, 'New Intent'] = '11'
        if row['Secondary Intent'] == 'Intent_4_3':
            df.at[index, 'New Intent'] = '12'
        if row['Secondary Intent'] == 'Intent_4_4':
            df.at[index, 'New Intent'] = '13'
        if row['Secondary Intent'] == 'Intent_5_2':
            df.at[index, 'New Intent'] = '14'
        if row['Secondary Intent'] == 'Intent_5_3':
            df.at[index, 'New Intent'] = '15'
        if row['Secondary Intent'] == 'Intent_5_4':
            df.at[index, 'New Intent'] = '16'

    df.drop(columns=['Secondary Intent'], inplace=True)
    df.rename(columns={'New Intent': 'Intent'}, inplace=True)
    
    return df

### 2.2 函数：离散特征One-Hot编码

In [7]:
def one_hot_encoding(df):
    # 离散特征变量使用One-Hot编码，通过DataFrame实现
    df = pd.get_dummies(df, columns=['Flight_Fleet_State', 'Flight_Fly_Mode', 'UAV_Selected_Now', 'UAV1_Task_State', 'UAV2_Task_State',
                                     'UAV3_Task_State', 'UAV4_Task_State', 'AOI', 'Intent'])
    return df

### 2.3 函数：连续特征归一化编码

In [8]:
def normalized_encoding(df):
    # 数据归一化-训练集
    df.loc[:, 'Hotas_Axis_X']  = (df['Hotas_Axis_X']+1)/2
    df.loc[:, 'Hotas_Axis_Y']  = (df['Hotas_Axis_Y']+1)/2
    df.loc[:, 'Hotas_Axis_Z']  = (df['Hotas_Axis_Z']+1)/2

    df.loc[:, 'Flight_Pos_X']  = (df['Flight_Pos_X']-df['Flight_Pos_X'].min())/(df['Flight_Pos_X'].max()-df['Flight_Pos_X'].min())
    df.loc[:, 'Flight_Pos_Y']  = (df['Flight_Pos_Y']-df['Flight_Pos_Y'].min())/(df['Flight_Pos_Y'].max()-df['Flight_Pos_Y'].min())
    df.loc[:, 'Flight_Speed']  = (df['Flight_Speed']-df['Flight_Speed'].min())/(df['Flight_Speed'].max()-df['Flight_Speed'].min())
    df.loc[:, 'Flight_Height']  = (df['Flight_Height']-df['Flight_Height'].min())/(df['Flight_Height'].max()-df['Flight_Height'].min())
    df.loc[:, 'Flight_Heading']  = (df['Flight_Heading']-df['Flight_Heading'].min())/(df['Flight_Heading'].max()-df['Flight_Heading'].min())

    df.loc[:, 'UAV1_Pos_X']  = df['UAV1_Pos_X']/3840
    df.loc[:, 'UAV1_Pos_Y']  = df['UAV1_Pos_Y']/2160
    df.loc[:, 'UAV2_Pos_X']  = df['UAV2_Pos_X']/3840
    df.loc[:, 'UAV2_Pos_Y']  = df['UAV2_Pos_Y']/2160
    df.loc[:, 'UAV3_Pos_X']  = df['UAV3_Pos_X']/3840
    df.loc[:, 'UAV3_Pos_Y']  = df['UAV3_Pos_Y']/2160
    df.loc[:, 'UAV4_Pos_X']  = df['UAV4_Pos_X']/3840
    df.loc[:, 'UAV4_Pos_Y']  = df['UAV4_Pos_Y']/2160

    df.loc[:, 'Target_Area_Pos_X']  = (df['Target_Area_Pos_X']-df['Target_Area_Pos_X'].min())/(df['Target_Area_Pos_X'].max()-df['Target_Area_Pos_X'].min())
    df.loc[:, 'Target_Area_Pos_Y']  = (df['Target_Area_Pos_Y']-df['Target_Area_Pos_Y'].min())/(df['Target_Area_Pos_Y'].max()-df['Target_Area_Pos_Y'].min())
    df.loc[:, 'Trapped_People_Pos_X']  = (df['Trapped_People_Pos_X']-df['Trapped_People_Pos_X'].min())/(df['Trapped_People_Pos_X'].max()-df['Trapped_People_Pos_X'].min())
    df.loc[:, 'Trapped_People_Pos_Y']  = (df['Trapped_People_Pos_Y']-df['Trapped_People_Pos_Y'].min())/(df['Trapped_People_Pos_Y'].max()-df['Trapped_People_Pos_Y'].min())

    df.loc[:, 'Gaze point X']  = df['Gaze point X']/3840
    df.loc[:, 'Gaze point Y']  = df['Gaze point Y']/2160
    
    return df

### 2.4 处理测试集数据

In [9]:
my_test = df_test.copy()
my_test = my_test.drop(columns=['Time', 'Primary Intent'])
my_test = secondary_intent_encoding(my_test)
my_test = one_hot_encoding(my_test)
my_test = normalized_encoding(my_test)
my_test = my_test[my_test['ID']==ID]

pd.set_option('display.max_columns', None)
my_test.head()

,User,ID,Hotas_Axis_X,Hotas_Axis_Y,Hotas_Axis_Z,Hotas_Btn_Create_Fleet,Hotas_Btn_Select_UAV1,Hotas_Btn_Select_UAV2,Hotas_Btn_Select_UAV3,Hotas_Btn_Select_UAV4,Hotas_Btn_Assign_Up,Hotas_Btn_Assign_Down,Hotas_Btn_Assign_Left,Hotas_Btn_Assign_Right,Hotas_Btn_Change_Mode,Hotas_Btn_Open_UAV_Sensor,Hotas_Btn_Open_Helicopter_Sensor,Hotas_Btn_Select_Normal,Hotas_Btn_Select_Fire,Hotas_Btn_Select_Person,Hotas_Btn_Submit_Result,Hotas_Btn_Area_Assign,Flight_Pos_X,Flight_Pos_Y,Flight_Speed,Flight_Height,Flight_Heading,Flight_Sensor,UAV1_Pos_X,UAV1_Pos_Y,UAV2_Pos_X,UAV2_Pos_Y,UAV3_Pos_X,UAV3_Pos_Y,UAV4_Pos_X,UAV4_Pos_Y,UAV_Sensor_State,Task_List_State,Target_Area_State,Target_Area_Pos_X,Target_Area_Pos_Y,Trapped_People_State,Trapped_People_Pos_X,Trapped_People_Pos_Y,Gaze point X,Gaze point Y,Flight_Fleet_State_0,Flight_Fleet_State_1,Flight_Fleet_State_2,Flight_Fly_Mode_0,Flight_Fly_Mode_1,UAV_Selected_Now_-1.0,UAV_Selected_Now_1.0,UAV_Selected_Now_2.0,UAV_Selected_Now_3.0,UAV_Selected_Now_4.0,UAV1_Task_State_0,UAV1_Task_State_1,UAV1_Task_State_2,UAV1_Task_State_3,UAV2_Task_State_0,UAV2_Task_State_1,UAV2_Task_State_2,UAV2_Task_State_3,UAV3_Task_State_0,UAV3_Task_State_1,UAV3_Task_State_2,UAV3_Task_State_3,UAV4_Task_State_0,UAV4_Task_State_1,UAV4_Task_State_2,UAV4_Task_State_3,AOI_0,AOI_1,AOI_2,AOI_3,AOI_4,AOI_5,AOI_6,Intent_00,Intent_01,Intent_02,Intent_03,Intent_04,Intent_05,Intent_06,Intent_07,Intent_08,Intent_09,Intent_10,Intent_11,Intent_12,Intent_13,Intent_14,Intent_15,Intent_16
1149664,10,50,0.5,0.5,1.0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0.510288,0.131829,0.0,0.516724,0.00001,False,0.495825,0.1839,0.534825,0.1839,0.495825,0.130567,0.534825,0.130567,False,True,False,0.0,0.0,False,0.0,0.0,0.455208,0.631481,True,False,False,True,False,True,False,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1149665,10,50,0.5,0.5,1.0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0.510288,0.131829,0.0,0.516724,0.00001,False,0.495825,0.1839,0.534825,0.1839,0.495825,0.130567,0.534825,0.130567,False,True,False,0.0,0.0,False,0.0,0.0,0.452604,0.634722,True,False,False,True,False,True,False,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1149666,10,50,0.5,0.5,1.0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0.510288,0.131829,0.0,0.516724,0.00001,False,0.495825,0.1839,0.534825,0.1839,0.495825,0.130567,0.534825,0.130567,False,True,False,0.0,0.0,False,0.0,0.0,0.455208,0.625000,True,False,False,True,False,True,False,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1149667,10,50,0.5,0.5,1.0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0.510288,0.131829,0.0,0.516724,0.00001,False,0.495825,0.1839,0.534825,0.1839,0.495825,0.130567,0.534825,0.130567,False,True,False,0.0,0.0,False,0.0,0.0,0.453906,0.629630,True,False,False,True,False,True,False,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1149668,10,50,0.5,0.5,1.0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0.510288,0.131829,0.0,

## 3. 定义DataLoader和DataSet

In [10]:
from torch.utils.data import DataLoader, Dataset
import numpy as np

class MyDataset(Dataset):
    # 实例化调用，数据集初始化
    def __init__(self, df, time_step):
        # 初始化输入输出
        x = []
        y = []
        num_intent_0 = 0
        
        # 根据test_id划分DataFrame
        grouped = df.groupby('ID')
        
        for _, group in grouped:
            # test_id放在最后
            origin_data = group.drop(columns=['ID', 'User'])
            # 将表格中的bool型转换为int型
            for column in origin_data.columns:
                if origin_data[column].dtype == bool:
                    origin_data[column] = origin_data[column].astype(int)
            # 将表格数据转换为Numpy
            dataset = (origin_data).to_numpy()
            for i in range(len(dataset) - time_step):
                input_seq = dataset[i:(i+time_step), 0:-17]
                output_seq = dataset[i+time_step-1, -17:]
                
                # if output_seq[0] == 1:
                    # num_intent_0 += 1
                # else:
                    # x.append(input_seq)
                    # y.append(output_seq[1:])
                
                x.append(input_seq)
                y.append(output_seq[1:])
        
        x = np.array(x)
        y = np.array(y)
        print(f'shape_input:{x.shape},shape_output:{y.shape}')
        print(f'delete Intent_0:{num_intent_0}')
        
        self.data_tensor = torch.tensor(x, dtype=torch.float32)
        self.label_tensor = torch.tensor(y, dtype=torch.float32)
    
    # 返回数据集的长度    
    def __len__(self):
        return self.data_tensor.shape[0]
    
    # 根据指定下标返回数据集内容    
    def __getitem__(self, i):
        # 读取数据
        data = self.data_tensor[i]
        # 读取标签
        label = self.label_tensor[i]

        return data, label  # 返回数据和标签

In [11]:
# 创建数据集
test_dataset = MyDataset(my_test, 20)
# 加载数据集
test_data = DataLoader(test_dataset, batch_size=1, shuffle=False)

print(f'输入张量：{test_dataset.data_tensor.shape} 输出张量：{test_dataset.label_tensor.shape}')

shape_input:(25005, 20, 77),shape_output:(25005, 16)
delete Intent_0:0
输入张量：torch.Size([25005, 20, 77]) 输出张量：torch.Size([25005, 16])


## 4. 测试集输入模型，验证准确率

In [12]:
# 定义相关参数：GPU、损失函数
device = 'cuda'
loss_function = nn.CrossEntropyLoss()

In [13]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# 测试指标
onnx_test_loss = 0
onnx_test_acc = 0
onnx_predicted_labels = []  # 模型结果标签，用于混淆矩阵
onnx_true_labels = []  # 模型真值标签，用于混淆矩阵

# 损失函数
onnx_loss_function = nn.CrossEntropyLoss()

for seq, labels in test_data:
    x,y = seq.to(device), labels.to(device)
    x = to_numpy(x)
    
    # 模型推理并输出
    ort_inputs = {ort_session.get_inputs()[0].name: x}
    ort_outs = ort_session.run(None, ort_inputs)
    y_pred = torch.tensor(ort_outs[0]).to(device)
    y_pred = torch.squeeze(y_pred, dim=1) # 去除额外维度
    single_loss = loss_function(y_pred, y)
    
        
    correct = torch.eq(y.argmax(dim=1), y_pred.argmax(dim=1))
    # print(f'y:{y.argmax(dim=1)},y_pred:{y_pred.argmax(dim=1)}')
    onnx_predicted_labels += y_pred.argmax(dim=1).tolist()
    onnx_true_labels += y.argmax(dim=1).tolist()
    
    onnx_test_acc += correct.sum().item()/ y.shape[0]
    onnx_test_loss += single_loss.item()

# 去平均
onnx_test_loss /= len(test_data)
onnx_test_acc /= len(test_data)

print(f'测试集平均损失：{onnx_test_loss}, \n测试集平均准确率：{onnx_test_acc}')

测试集平均损失：0.07240484751507775, 
测试集平均准确率：0.8672665466906618


## 5. 将识别后的结果导出为表格

In [14]:
len(onnx_predicted_labels)

25005

In [15]:
df_test.shape

(233086, 57)

In [16]:
def add_intention_recognition_results(df, predicted_labels):
    df.drop(columns=['Secondary Intent'], inplace=True)
    df['Secondary Intent'] = None
    df = df.reset_index(drop=True)
    for index, row in df.iterrows():
        if index >= 20:
            df.at[index, 'Secondary Intent'] = predicted_labels[index - 20]
    
    return df

df_test_final = df_test.copy()
df_test_final = df_test_final[df_test_final['ID']==ID]
df_test_final = add_intention_recognition_results(df_test_final, onnx_predicted_labels)
df_test_final.to_csv(f'Temporary Data/{User}_predicted_data.csv')